# Ray crash course - Distributed HPO with Ray Tune and XGBoost-Ray

© 2019-2022, Anyscale. All Rights Reserved

This demo introduces **Ray tune's** key concepts using a classification example. This example is derived from [Hyperparameter Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

Three simple steps:

 1. Setup your config space and define your trainable and objective function
 2. Use Tune to execute your training hyperparameter sweep, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#summary) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine or analyse the results returned
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Understanding Hyperparameter Tuning](https://github.com/anyscale/academy/blob/main/ray-tune/02-Understanding-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=False

/usr/local/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-ray-core-tutorial")
    else:
        ray.init()

2022-04-12 20:13:23,076	INFO services.py:1460 -- View the Ray dashboard at http://127.0.0.1:8268


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial with a set of unique config from the search space
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(train_func_model pid=77912) 2022-04-12 20:13:56,513	INFO main.py:1509 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 4.62 seconds (0.96 pure XGBoost training time).
2022-04-12 20:13:56,658	INFO tune.py:702 -- Total run time: 25.97 seconds (20.70 seconds for the tuning loop).


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.01917095487374944, 'subsample': 0.8056698691344234, 'max_depth': 8}


In [8]:
analysis.results_df.head(5)

/Users/jules/git-repos/ray/python/ray/tune/analysis/experiment_analysis.py:303: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,warmup_time,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
b1e45_00000,0.550525,0.008787,0.007021,True,None,None,10,f3511dccfa8b4c368b527741724fab8f,2022-04-12_20-13-44,1649819624,...,0.003291,"0_eta=0.0192,max_depth=8,subsample=0.8057",approx,binary:logistic,"[logloss, error]",0.019171,0.805670,8,1,1
b1e45_00001,0.680776,0.024605,0.004166,True,None,None,10,c2a612f1f883481298d2768588b9c198,2022-04-12_20-13-47,1649819627,...,0.003659,"1_eta=0.0015,max_depth=3,subsample=0.8611",approx,binary:logistic,"[logloss, error]",0.001468,0.861080,3,1,1
b1e45_00002,0.687816,0.047452,0.011657,True,None,None,10,6a5312d5fe7b433080ec87efb9354d3a,2022-04-12_20-13-53,1649819633,...,0.003291,"2_eta=0.0007,max_depth=2,subsample=0.9303",approx,binary:logistic,"[logloss, error]",0.000661,0.930260,2,1,1
b1e45_00003,0.394486,0.026362,0.003602,True,None,None,10,c3b5f34bf7e74887b2cd2c781a9da35d,2022-04-12_20-13-56,1649819636,...,0.003035,"3_eta=0.0517,max_depth=7,subsample=0.5911",approx,binary:logistic,"[logloss, error]",0.051749,0.591094,7,1,1


---

In [9]:
ray.shutdown()

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)